#ANALYSE DE SENTIMENTS

Pour cette certification vous aurez à étudier un corpus de données extrait du **Large Movie Review Dataset v1.0**. Ce corpus est composé de critiques en anglais de films tirées de **IMDb** et préprocessées :
- Le Corpus est déjà séparé en `train/` et `test/`. 
- Les commentaires positifs sont dans un dossier `pos/`, les négatifs dans `neg/`.
- Chaque critique est dans un fichier texte distinct, sur une ligne, sans retour chariot.

L'objectif de ce TP de certification est de produire des modèles prédictifs capables de donner la polarité d'une critique de film, ainsi que de commenter les résultats obtenus.

Dans un premier temps, il vous sera demandé de **collecter les données**, dans un second d'utiliser une **représentation en sac de mots pondérés par TF-IDF** ainsi qu'un algorithme de machine learning adapté. Enfin, vous devrez utiliser un **réseaux profond récurrent** avec et sans **word embeddings** déjà appris.

Pour des raisons évidentes de **temps limité**, ce TP n'a pas pour but de vous faire chercher le meilleur modèle avec les meilleures performances. Les **méta-paramètres** d'apprentissage **vous seront toujours fournis** dans le but de rendre les executions raisonnablement courtes.

## Récupération des données

Les données se trouvent dans le repository github suivant : https://github.com/nzmonzmp/sentiment-aclimdb.git

In [0]:
! git clone https://github.com/nzmonzmp/sentiment-aclimdb.git

Cloning into 'sentiment-aclimdb'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 78582 (delta 0), reused 3 (delta 0), pack-reused 78579
Receiving objects: 100% (78582/78582), 36.33 MiB | 10.23 MiB/s, done.
Resolving deltas: 100% (28972/28972), done.
Checking out files: 100% (50002/50002), done.


## Ressource extérieure 

Cette ressource est nécéssaire pour la Dernière partie de cet examen. Son **téléchargement étant un peu long (~10mn)**, si vous voulez commencer par lancer cette cellule pendant que vous parcourez l'intégralité de l'examen, vous gagnerez de précieuses minutes pour plus tard.

In [0]:
! wget http://nlp.stanford.edu/data/glove.6B.zip
! unzip glove.6B.zip

def glove_filename(embedding_dim):
  if embedding_dim in {50, 100, 200, 300}:
    return "glove.6B.{}d.txt".format(embedding_dim)
  else:
    raise ValueError("EMBEDDING_DIM must be in {50, 100, 200, 300}")

--2020-05-25 07:16:58--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-25 07:16:58--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-25 07:16:59--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

#Partie 1 : COLLECTE

Après un rapide coup d'oeil aux données dans le dossier `sentiment-aclimdb/`, **implémentez une fonction** qui va vous permettre d'instancier les variables suivantes :
- `X_train_raw` et `X_test_raw`, des listes contenant une critique par indice
- `Y_train` et `Y_test`, des listes contenant la cible sous forme d'entier (`0` : négatif , `1` : positif)
- `X_train_names` et `X_test_names`, les listes contenant les chemins des fichiers correspondant à chaque exemple

Instanciez-les.

In [0]:
from pathlib import Path
from sklearn.utils import shuffle

def get_corpus(directory: Path):
    reviews = []
    labels = []
    file_names = []
    for p in ['neg','pos']:
      curr_dir = directory / p
      target = 1 if p == "pos" else 0
      for file_path in curr_dir.iterdir():
        text = file_path.read_text(encoding="utf8")
        reviews.append(text)
        labels.append(target)
        file_names.append(str(file_path))

    return shuffle(reviews, labels, file_names, random_state=817328462)
        
X_train_raw, Y_train, X_train_names = get_corpus(Path("sentiment-aclimdb") / "train")
X_test_raw , Y_test,  X_test_names  = get_corpus(Path("sentiment-aclimdb") / "test")

#Partie 2 : TF-IDF

## Représentation

Transformez `X_train_raw` et `X_test_raw` à l'aide de [`TfidfVectorizer`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) de la librairie **scikit-learn**.

On ne conservera que **1000 mots** dans le dictionnaire.

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=1000)
vectorizer.fit(X_train_raw)
X_train = vectorizer.transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)

## Modélisation

Choisissez un modèle **adapté** à la classification de représentations TF-IDF.

Apprenez-le et calculez son score d'accuracy en apprentissage comme en test.

In [0]:
from sklearn import svm
clf = svm.LinearSVC()
clf.fit(X_train,Y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [0]:
print("Accuracy sur train : {:.3f}".format(clf.score(X_train,Y_train)))
print("Accuracy sur test  : {:.3f}".format(clf.score(X_test,Y_test)))

Accuracy sur train : 0.881
Accuracy sur test  : 0.864


## Analyse d'erreur

Affichez les 5 critiques positives les plus mal classées de la base de test.

Idem pour les négatives les mieux classées.

In [0]:
import numpy as np

Y_pred = clf.decision_function(X_test)

ind_pos = np.where(Y_test == 1)[0]
ind_neg = np.where(Y_test == 0)[0]

worst_pos = ind_pos[np.argsort(Y_pred[ind_pos])[:5]]
best_neg = ind_neg[np.argsort(-Y_pred[ind_neg])[:5]]

print("Critiques prédites comme étant très négative mais avec un label cible positif :")
for i in worst_pos:
  print("  {:.3f} : {}".format(Y_pred[i], X_test_raw[i]))
print()
print("--------------------------------------------------------------------------------")
print()
print("Critiques prédites comme étant très positives mais avec un label cible négatif :")
for i in best_neg:
  print("  {:.3f} : {}".format(Y_pred[i], X_test_raw[i]))


Critiques prédites comme étant très négative mais avec un label cible positif :
  -2.208 : It's not Citizen Kane, but it does deliver. Cleavage, and lots of it.Badly acted and directed, poorly scripted. Who cares? I didn't watch it for the dialog.
  -1.908 : David Morse and Andre Braugher are very talented actors, which is why I'm trying so hard to support this program. Unfortunately, an irrational plot, and very poor writing is making it difficult for me. I'm hoping that the show gets a serious overhaul, or that the actors find new projects that are worthy of them.
  -1.881 : **SPOILERS AHEAD**It is really unfortunate that a movie so well produced turns out to besuch a disappointment. I thought this was full of (silly) clichés andthat it basically tried to hard. To the (American) guys out there: how many of you spend yourtime jumping on your girlfriend's bed and making monkeysounds? To the (married) girls: how many of you have suddenlygone from prudes to nymphos overnight--but not wit

## Question

Les représentations en sac de mots pondérés par TF-IDF possèdent des limitations quand il s'agit de capter la polarité d'un texte. 

**Lesquelles pouvez-vous citer ?**

## Réponse

- Perte de l'articulation des mots dans les phrases.
- Quand plusieurs phrases constituent une argumentation, il est impossible avec une représentation TF-IDF de savoir quelle parties du discours sont positives/négatives.

# Partie 3 : UTILISATIONS DE RÉSEAUX RÉCURRENT SUR DES SÉQUENCES DE MOTS



## Préparation du dictionnaire et des séquences

À l'aide de l'outil [`tensorflow.keras.preprocessing.text.Tokenizer`](https://keras.io/api/preprocessing/text/#tokenizer-class) :
- Constituez un dictionnaire sur le Corpus `X_train_raw`
- Quelle est la taille du vocabulaire ?
- Quelle est la taille maximum, en nombre de mots, d'une critique ?

In [0]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(X_train_raw)

In [0]:
vocab_size = len(tokenizer_obj.word_index) + 1
max_length = max(len(s.split()) for s in X_train_raw)
print("Taille du vocabulaire : {}".format(vocab_size))
print("Taille de la plus grande séquence de mot : {}".format(max_length))

Taille du vocabulaire : 89663
Taille de la plus grande séquence de mot : 2450


À l'aide des outils [`tensorflow.keras.preprocessing.text.Tokenizer`](https://keras.io/api/preprocessing/text/#tokenizer-class) et [`tensorflow.keras.preprocessing.sequence.pad_sequences`](https://keras.io/api/preprocessing/timeseries/#padsequences-function) :
- Transformez `X_train_raw` et `X_test_raw` en séquences d'indices de mots dans un dictionnaire.
- Effectuez l'opération de **padding** sur les séquences afin qu'elles aient une taille raisonnable pour être traitées par un GRU bidirectionnel.
- Justifiez vos choix pour les options `maxlen` et `truncating` de la fonction `pad_sequences`
- Stockez les séquences obtenues dans `X_train_pad` et `X_test_pad`
- Pour des raisons de compatibilité pour la suite, transformez `X_train_pad`, `X_test_pad`, `Y_train`, `Y_test` en `numpy.ndarray` (en utilisant la fonction `numpy.array`)


In [0]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
max_length = 150

X_train_tokens = tokenizer_obj.texts_to_sequences(X_train_raw)
X_test_tokens = tokenizer_obj.texts_to_sequences(X_test_raw)

X_train_pad = pad_sequences(X_train_tokens, maxlen=max_length, truncating="pre")
X_test_pad = pad_sequences(X_test_tokens, maxlen=max_length, truncating="pre")

X_train_pad = np.array(X_train_pad)
X_test_pad = np.array(X_test_pad)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

## Justifications

- `maxlen=150` : les GRU bidirectionnels conservent une information de qualité pour des séquences de mots de cet ordre de grandeur de longueur.
- `truncating="pre"` : on ne conserve que la fin de la critique, plus susceptible de contenir une information pertinente quand on fait une critique, i.e la conclusion

## Modélisation

Construisez un modèle avec pour caractéristiques :
  - Une couche d'embeddings de taille 300
  - Une couche de GRU bidirectionnels :
    - de taille `16`
    - une initialisation **des** matrices de poids orthogonales
    - un paramètre de dropout à `0.2` pour les parties forward et récurrentes
  - Une couche de réseau de neurones à activation `softmax` qui prend en entrée la dernière sortie de la couche [`GRU`](https://keras.io/api/layers/recurrent_layers/gru/)
  - Une fonction de perte basée sur l'**entropie croisée**
  - L'optimiseur `adam`
  - l'`accuracy` comme métrique d'évaluation

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU , Bidirectional
EMBEDDING_DIM = 300
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(Bidirectional(GRU(16, kernel_initializer="orthogonal", recurrent_initializer="orthogonal", dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

### Questions

Quel est l'intêret de :
- l'initialisation **orthogonale** des matrices de poids ?
- la version **bidirectionnel** des GRU ?
- du **dropout** ?

### Réponses

- L'initialisation orthogonale permet d'**éviter l'explosion des gradients** dans les réseaux récurrents
- Les RNN bidirectionnels, permetttent de traiter plus efficacement des **séquences longues** tout en améliorant la qualité de la **contextualisation** de l'information
- Le dropout est une technique de **régularisation** et donc essaye d'empêcher le suraprentiage

## Apprentissage

Apprenez votre modèle sur `X_train_pad` :
- avec des batch de taille `256`
- pendant `10` itérations
- en utilisant un split `0.3` (30%) de la base d'apprentissage pour validation


In [0]:
model.fit(X_train_pad, Y_train, batch_size=256, epochs=10, validation_split=0.3)

Epoch 1/10
69/69 [==============================] - 119s 2s/step - loss: 0.6325 - accuracy: 0.6403 - val_loss: 0.5246 - val_accuracy: 0.7287
Epoch 2/10
69/69 [==============================] - 117s 2s/step - loss: 0.3467 - accuracy: 0.8475 - val_loss: 0.3618 - val_accuracy: 0.8479
Epoch 3/10
69/69 [==============================] - 117s 2s/step - loss: 0.1695 - accuracy: 0.9371 - val_loss: 0.3894 - val_accuracy: 0.8555
Epoch 4/10
69/69 [==============================] - 117s 2s/step - loss: 0.0821 - accuracy: 0.9737 - val_loss: 0.4238 - val_accuracy: 0.8553
Epoch 5/10
69/69 [==============================] - 117s 2s/step - loss: 0.0443 - accuracy: 0.9866 - val_loss: 0.4939 - val_accuracy: 0.8439
Epoch 6/10
69/69 [==============================] - 117s 2s/step - loss: 0.0242 - accuracy: 0.9938 - val_loss: 0.5230 - val_accuracy: 0.8503
Epoch 7/10
69/69 [==============================] - 118s 2s/step - loss: 0.0145 - accuracy: 0.9966 - val_loss: 0.5881 - val_accuracy: 0.8507
Epoch 8/10
69

## Évaluation

Évaluez les performances de votre modèle sur la base de test.

In [0]:
test_loss, test_accuracy = model.evaluate(X_test_pad, Y_test)
print("Accuracy sur la base de test : {:.3f}".format(test_accuracy))

782/782 [==============================] - 71s 90ms/step - loss: 0.7814 - accuracy: 0.8312
Accuracy sur la base de test : 0.831


### Question

Que peut-on conclure de l'apprentissage de notre modèle ? Justifiez

### Réponse

D'après l'évolution de l'accuracy sur train et sur valid, on observe très nettement un phénomène de surapprentissage :
- train accuracy > 99%
- valid accuracy < 85%

# Partie 4 : UTILISATION DE WORD EMBEDDINGS PRÉ-APPRIS

Afin d'obtenir de meilleurs résultats, vous allez utiliser les embeddings pré-appris  **GloVe**¹, plutôt que d'apprendre des embeddings spécifiques comme précédemment.



¹ Jeffrey Pennington, Richard Socher, and Christopher D. Manning. [*GloVe: Global Vectors for Word Representation*](https://nlp.stanford.edu/pubs/glove.pdf), 2014.


## Réutilisation des Word Embeddings **GloVe**

Après un rapide coup d'oeil aux fichiers `glove.6B.*.txt` :
- Créez un dictionnaire `embeddings_dic` avec les mots pour clés et leur vecteurs de `float32` respectifs en valeur.
- Créez un layer `embedding_layer` de type [`tensorflow.keras.layers.Embedding`](https://keras.io/api/layers/core_layers/embedding/) :
  - Initialisez-le avec les embeddings GloVe. 
  - Initialisez les mots de notre vocabulaire qui ne seraient pas dans GloVe avec le vecteur nul
  - Utilisez le flag approprié pour empêcher le changement de ces embeddings pendant l'apprentissage

In [0]:
embeddings_dic = {}
f = open(glove_filename(EMBEDDING_DIM))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_dic[word] = coefs
f.close()

embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
cpt = 0
for word, i in tokenizer_obj.word_index.items():
    embedding_vector = embeddings_dic.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        cpt = cpt+1
print("found "+str(cpt)+" embeddings over "+str(vocab_size)+" words in vocab")

embedding_layer = Embedding(vocab_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False)

## Modélisation, Apprentissage et Évaluation

- Créez un modèle identique à celui de la partie 3, à ceci près que la couche d'**Embeddings** est celle que l'on vient d'instancier à partir de **GloVe**
- Entrainez ce modèle avec les mêmes paramètres que dans la partie 3
- Évaluez-le sur les données de test

In [0]:
model = Sequential()
model.add(embedding_layer)
model.add(Bidirectional(GRU(16, kernel_initializer='orthogonal' , recurrent_initializer='orthogonal', dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2 , activation ='softmax'))

model.compile(loss='sparse_categorical_crossentropy' , optimizer='adam', metrics=["accuracy"])


In [0]:
model.fit(X_train_pad, Y_train, batch_size=256, epochs=10, validation_split=0.3)

Epoch 1/10
69/69 [==============================] - 61s 889ms/step - loss: 0.6427 - accuracy: 0.6234 - val_loss: 0.5739 - val_accuracy: 0.7083
Epoch 2/10
69/69 [==============================] - 60s 875ms/step - loss: 0.4872 - accuracy: 0.7663 - val_loss: 0.4497 - val_accuracy: 0.7971
Epoch 3/10
69/69 [==============================] - 60s 864ms/step - loss: 0.4176 - accuracy: 0.8133 - val_loss: 0.3962 - val_accuracy: 0.8219
Epoch 4/10
69/69 [==============================] - 59s 860ms/step - loss: 0.3818 - accuracy: 0.8351 - val_loss: 0.3783 - val_accuracy: 0.8341
Epoch 5/10
69/69 [==============================] - 59s 856ms/step - loss: 0.3559 - accuracy: 0.8479 - val_loss: 0.3654 - val_accuracy: 0.8439
Epoch 6/10
69/69 [==============================] - 59s 856ms/step - loss: 0.3360 - accuracy: 0.8561 - val_loss: 0.3463 - val_accuracy: 0.8501
Epoch 7/10
69/69 [==============================] - 59s 861ms/step - loss: 0.3228 - accuracy: 0.8638 - val_loss: 0.3397 - val_accuracy: 0.8532

In [0]:
model.evaluate(X_test_pad, Y_test)

782/782 [==============================] - 49s 63ms/step - loss: 0.3195 - accuracy: 0.8658


[0.3194841742515564, 0.8658000230789185]

## Question

Que peut-on conclure de l'apprentissage de notre modèle ? Justifiez

## Réponse

D'après l'évolution de l'accuracy sur train et sur valid, le phénomène de surapprentissage à quasiement disparu : les performances en train, valid et test sont toutes dans le même ordre de grandeur.